<a href="https://colab.research.google.com/github/meparth/StyleTransferofText/blob/master/Copy_of_sentimental_style_transfer_in_text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## import things


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

In [ ]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch.nn.functional as F
import matplotlib.pyplot as plt
import random
import numpy as np
from collections import Counter
%matplotlib inline

In [ ]:
import gensim
from __future__ import print_function
from gensim.models import KeyedVectors
import numpy as np
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

## get the data

In [ ]:
# mount drive of higloomybunny
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


**Cricket** 

positive: 566
negative: 2152

**Restaurant** 

positive: 1221
negative: 472





In [ ]:
# df = pd.read_csv('drive/My Drive/data/padataset.csv') # prothom-alo         pos-neu-neg
# df = pd.read_excel('drive/My Drive/data/facebook_comments.xlsx')     # sweetyr dataset
df = pd.read_excel('drive/My Drive/data/Cricket.xlsx')     # cricket           pos-neg
# df = pd.read_excel('drive/My Drive/data/Restaurant.xlsx')     # restaurant       pos-neg



""" for padataset """
# negative = df[df['value'] < 0]
# positive = df[df['value'] > 0]
# neutral  = df[df['value'] == 0]

""" for cricket and restaurant """
positive = df[df['Polarity']=='positive']
negative = df[df['Polarity']=='negative']



In [ ]:
positive.shape

(1221, 4)

In [ ]:
""" padataset """
# negative2 = set (negative.data)
# positive2 = set (positive.data)
# neutral2 = set(neutral.data)

In [ ]:
""" facebook comments """
# positive2 = set(positive.comment)
# negative2 = set(negative.comment)

In [ ]:
""" cricket and restaurant dataset """ 
negative2 = set (negative.Text)
positive2 = set (positive.Text)

In [ ]:
type(negative2)

set

In [ ]:
""" positive and negative """
pos = []
cnt = 0
for s in positive2:
    if len(s.split()) <= 20:
        pos.append(s)
        cnt = cnt + 1

data_positive = pos

neg = []
cnt = 0
for s in negative2:
    if len(s.split()) <= 20:
        neg.append(s)
        cnt = cnt + 1

data_negative = neg

In [ ]:
""" neutral data """ 

neu = []
cnt = 0
for s in neutral2:
    if len(s.split()) <= 20:
        neu.append(s)
        cnt = cnt + 1

data_neutral = neu

In [ ]:
print(len(data_positive), len(data_negative))

4016 3822


In [ ]:
""" 
pa: 
cricket:    positive*4
restaurant: negative*3
facebook: 

"""
data_positive = data_positive*8
data_negative = data_negative*2
# data_neutral = data_neutral*1


In [ ]:
len(data_positive)

921

## embedding

In [ ]:
bn_model = KeyedVectors.load_word2vec_format('drive/My Drive/data/cc.bn.300.vec')

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
# Limit number of tokens 
limit = 500
vector_dim = 300

# Getting tokens and vectors
words = []
embedding = np.array([])
i = 0
for word in bn_model.vocab:
    # Break the loop if limit exceeds 
    if i == limit: break

    # Getting token 
    words.append(word)

    # Appending the vectors 
    embedding = np.append(embedding, bn_model[word])

    i += 1

embedding = embedding.reshape(limit, vector_dim)

common crawl vocabulary(bn_model.vocab): 1468578

In [ ]:
len(bn_model.vocab)

1468578

In [ ]:
weight = torch.FloatTensor(bn_model.vectors)

In [ ]:
USE_CUDA = torch.cuda.is_available()
USE_CUDA

True

## sampling

In [ ]:
import random
""" 
cricket: 1900
restaurant: 920
"""

randomsample = 3500

dataPOS = random.sample(data_positive,randomsample) 
dataNEG = random.sample(data_negative,randomsample)
# dataNEU = random.sample(neutral2, 2000)

In [ ]:
import nltk
nltk.download('punkt')
SEQ_LENGTH = 22
trainPOS = []
trainNEG = []
trainNEU = []

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
""" positive and negative """

for t in dataPOS:
    t0 = t
    t0 = t0.replace("<br>","")
    t0 = t0.replace("/","")
    token0 = nltk.word_tokenize(t0)
   
    
    if len(token0)>=SEQ_LENGTH:
        token0= token0[:SEQ_LENGTH-1]
    token0.append("<EOS>")

    while len(token0)<SEQ_LENGTH:
        token0.append('<PAD>')
    
    trainPOS.append([token0,token0,1])

for t in dataNEG:
    t0 = t
    t0 = t0.replace("<br>","")
    t0 = t0.replace("/","")
    token0 = nltk.word_tokenize(t0)
   
    
    if len(token0)>=SEQ_LENGTH:
        token0= token0[:SEQ_LENGTH-1]
    token0.append("<EOS>")

    while len(token0)<SEQ_LENGTH:
        token0.append('<PAD>')
    
    trainNEG.append([token0,token0,0])

In [ ]:
""" for neutral data """ 
for t in dataNEU:
    t0 = t
    t0 = t0.replace("<br>","")
    t0 = t0.replace("/","")
    token0 = nltk.word_tokenize(t0)
   
    
    if len(token0)>=SEQ_LENGTH:
        token0= token0[:SEQ_LENGTH-1]
    token0.append("<EOS>")

    while len(token0)<SEQ_LENGTH:
        token0.append('<PAD>')
    
    trainNEU.append([token0,token0,0])

NameError: ignored

In [ ]:
word2index={"<PAD>":0,"<SOS>":1,"<EOS>":2,"<UNK>":3}

for t in trainPOS:
    for token in t[0]:
        if token not in word2index:
            word2index[token]=len(word2index)



for t in trainNEG:
    for token in t[0]:
        if token not in word2index:
            word2index[token]=len(word2index)


# for t in trainNEU:
#     for token in t[0]:
#         if token not in word2index:
#             word2index[token]=len(word2index)

index2word = {v:k for k,v in word2index.items()}

In [ ]:
word2index

{'<PAD>': 0,
 '<SOS>': 1,
 '<EOS>': 2,
 '<UNK>': 3,
 'বিপিএলে': 4,
 'রাজশাহী': 5,
 'ভাল': 6,
 'খেলতে': 7,
 'পারেনি।': 8,
 'এজন্য': 9,
 'আমরা': 10,
 'সমর্থকরা': 11,
 'হতাশ': 12,
 'নই।': 13,
 'শ্রীলংকান': 14,
 'ব্যাটসম্যান': 15,
 'রোশন': 16,
 'সিলভা': 17,
 'রাব্বি': 18,
 'মানের': 19,
 'পেসার': 20,
 '...': 21,
 'রনিকেও': 22,
 'নিতে': 23,
 'পারত।': 24,
 'সাব্বিরের': 25,
 'শেষের': 26,
 'ছক্কা': 27,
 'টা': 28,
 'অসাধারণ': 29,
 'লেগেছে।': 30,
 'প্রথম': 31,
 'উইকেট': 32,
 'আব্দুর': 33,
 'রাজ্জাকই': 34,
 'নিয়েছে': 35,
 '.': 36,
 'ইজ্জত': 37,
 'রক্ষা': 38,
 'হইছে': 39,
 '___যাইহোক': 40,
 'অসংখ্য': 41,
 'ধন্যবাদ': 42,
 'মমিনুল': 43,
 'কে': 44,
 'মিরপুর': 45,
 'উইকেটে': 46,
 'বাহাতি': 47,
 'সাকিব': 48,
 'এর': 49,
 'বেশি': 50,
 'তাই': 51,
 'দলে': 52,
 'রাজ্জাক': 53,
 'মুমিনুলের': 54,
 'পর': 55,
 ',': 56,
 'রাজের': 57,
 'রাজসিক': 58,
 'প্রত্যাবর্তন।': 59,
 'বশ্যই': 60,
 'বাংলাদেশ': 61,
 'এই': 62,
 'সিরিজে': 63,
 'শ্রিলংকা': 64,
 'প্রতিটা': 65,
 'ম্যাচে': 66,
 'হারিয়ে': 67,
 'হাতুরুকে': 68,
 'একটা': 

In [ ]:
def prepare_sequence(seq, to_ix):
    idxs = list(map(lambda w: to_ix[w] if w in to_ix.keys() else to_ix["<UNK>"], seq))
    tensor = Variable(torch.LongTensor(idxs)).cuda() if USE_CUDA else Variable(torch.LongTensor(idxs))
    return tensor


flatten = lambda l: [item for sublist in l for item in sublist]

In [ ]:
flatten

<function __main__.<lambda>>

## Encoder

In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size,latent_size=10,n_layers=1):
        super(Encoder, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        self.Wmu= nn.Linear(hidden_size,latent_size)
        self.Wsigma = nn.Linear(hidden_size,latent_size)
        self.embedding = nn.Embedding.from_pretrained(weight)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers,batch_first=True)
    
    def reparametrize(self, mu, log_var):
        """"z = mean + eps * sigma where eps is sampled from N(0, 1)."""
        eps = Variable(torch.randn(mu.size(0), mu.size(1))).cuda() if USE_CUDA else Variable(torch.randn(mu.size(0), mu.size(1)))
        z = mu + eps * torch.exp(log_var/2)    # 2 for convert var to std
        return z
    
    def forward(self, input,train=True):
        hidden = Variable(torch.zeros(self.n_layers, input.size(0), self.hidden_size)).cuda() if USE_CUDA else Variable(torch.zeros(self.n_layers, input.size(0), self.hidden_size))
        
        embedded = self.embedding(input)
        output, hidden = self.gru(embedded, hidden)
        mu = self.Wmu(hidden[-1])
        log_var = self.Wsigma(hidden[-1])
        z = self.reparametrize(mu, log_var)
        
        return z,mu,log_var

## Generator

In [ ]:
class Generator(nn.Module):
    def __init__(self, hidden_size, output_size,latent_size=10,code_size=2, n_layers=1):
        super(Generator, self).__init__()
        
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        #self.Wz = nn.Linear(latent_size+code_size,hidden_size)
        self.Wz = nn.Linear(latent_size,hidden_size)
        self.tanh = nn.Tanh()
        self.embedding = nn.Embedding.from_pretrained(weight)

        #self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size, self.n_layers,batch_first=True)
        self.out = nn.Linear(self.hidden_size, self.output_size)
        
    def forward(self, input,latent,code,lengths,seq_length,training=True):
        

        embedded = self.embedding(input)
        #embedded = self.dropout(embedded)
       
        # h0
        #latent_code = torch.cat((latent,code),1) # z,c
        #hidden = self.tanh(self.Wz(latent_code)).view(self.n_layers,input.size(0),-1) 
        hidden = self.tanh(self.Wz(latent)).view(self.n_layers,input.size(0),-1) 
        decode=[]
        # Apply GRU to the output so far
        for i in range(seq_length):
            
            _, hidden = self.gru(embedded, hidden)
            score = self.out(hidden.view(hidden.size(0)*hidden.size(1),-1))
            softmaxed = F.log_softmax(score)
            decode.append(softmaxed)
            _,input = torch.max(softmaxed,1)
            embedded = self.embedding(input.unsqueeze(1))
            #embedded = self.dropout(embedded)
        
        # time-step column-wise concat, reshape!!
        scores = torch.cat(decode,1)
        
        return scores.view(input.size(0)*seq_length,-1)

## Discriminator

In [ ]:
class  Discriminator(nn.Module):
    
    def __init__(self, embed_num,embed_dim,class_num,kernel_num,kernel_sizes,dropout):
        super(Discriminator,self).__init__()
        #self.args = args
        
        V = embed_num # num of vocab
        D = embed_dim # dimenstion of word vector
        C = class_num # num of class
        Ci = 1
        Co = kernel_num # 100
        Ks = kernel_sizes # [3,4,5]

        self.embed = nn.Embedding(V, D)
        #self.convs1 = [nn.Conv2d(Ci, Co, (K, D)) for K in Ks]
        self.convs1 = nn.ModuleList([nn.Conv2d(Ci, Co, (K, D)) for K in Ks])
        
        # kernal_size = (K,D) 
        
        '''
        self.conv13 = nn.Conv2d(Ci, Co, (3, D))
        self.conv14 = nn.Conv2d(Ci, Co, (4, D))
        self.conv15 = nn.Conv2d(Ci, Co, (5, D))
        '''
        self.dropout = nn.Dropout(dropout)
        self.fc1 = nn.Linear(len(Ks)*Co, C)

    def conv_and_pool(self, x, conv):
        x = F.relu(conv(x)).squeeze(3) #(N,Co,W)
        x = F.max_pool1d(x, x.size(2)).squeeze(2)
        return x


    def forward(self, x,train=True):
        x = self.embed(x) # (N,W,D)
        
        #if self.args.static:
        #    x = Variable(x)

        x = x.unsqueeze(1) # (N,Ci,W,D)

        x = [F.relu(conv(x)).squeeze(3) for conv in self.convs1] #[(N,Co,W), ...]*len(Ks)


        x = [F.max_pool1d(i, i.size(2)).squeeze(2) for i in x] #[(N,Co), ...]*len(Ks)

        x = torch.cat(x, 1)

        '''
        x1 = self.conv_and_pool(x,self.conv13) #(N,Co)
        x2 = self.conv_and_pool(x,self.conv14) #(N,Co)
        x3 = self.conv_and_pool(x,self.conv15) #(N,Co)
        x = torch.cat((x1, x2, x3), 1) # (N,len(Ks)*Co)
        '''
        if train:
            x = self.dropout(x) # (N,len(Ks)*Co)
        logit = self.fc1(x) # (N,C)
        return logit

In [ ]:
HIDDEN_SIZE = 300
LATENT_SIZE = 10
CODE_SIZE = 2
BATCH_SIZE=32
STEP=300
KTA = 0.0
LEARNING_RATE=0.001

In [ ]:
encoder =  Encoder(len(word2index), HIDDEN_SIZE,LATENT_SIZE, 2)
generatorPOS = Generator(HIDDEN_SIZE,len(word2index),LATENT_SIZE,CODE_SIZE)
generatorNEG = Generator(HIDDEN_SIZE,len(word2index),LATENT_SIZE,CODE_SIZE)
# generatorNEU = Generator(HIDDEN_SIZE,len(word2index),LATENT_SIZE,CODE_SIZE)


# discriminator = Discriminator(len(word2index),100,2,30,[3,4,5],0.8)
if USE_CUDA:
    encoder = encoder.cuda()
    generatorPOS = generatorPOS.cuda()
    generatorNEG = generatorNEG.cuda()
    # generatorNEU = generatorNEU.cuda()


    # discriminator = discriminator.cuda()
    
Recon = nn.CrossEntropyLoss(ignore_index=0)


enc_optim= torch.optim.Adam(encoder.parameters(), lr=LEARNING_RATE)
gen_optimPOS = torch.optim.Adam(generatorPOS.parameters(),lr=LEARNING_RATE)
gen_optimNEG = torch.optim.Adam(generatorNEG.parameters(),lr=LEARNING_RATE)
# gen_optimNEU = torch.optim.Adam(generatorNEU.parameters(),lr=LEARNING_RATE)


# dis_optiom = torch.optim.Adam(discriminator.parameters(),lr=LEARNING_RATE)

In [ ]:
def getBatch(batch_size,train_data):
    random.shuffle(train_data)
    sindex=0
    eindex=batch_size
    while eindex < len(train_data):
        batch = train_data[sindex:eindex]
        x,y,c = zip(*batch)
        x,y,c = torch.cat(x),torch.cat(y),torch.cat(c)
        temp = eindex
        eindex = eindex+batch_size
        sindex = temp
        
        yield (x,y,c)

## train

In [ ]:
def TRAIN(train, encoder, generator, enc_optom, gen_optim):
    
    elboloss = []
    reconloss = []
    kldloss = []
    
    train_x=[]
    train_y=[]
    code_labels=[]
    lengths=[]
    for tr in train:
        temp = prepare_sequence(tr[0], word2index)
        temp = temp.view(1,-1)
        train_x.append(temp)

        temp2 = prepare_sequence(tr[1],word2index)
        temp2 = temp2.view(1,-1)
        train_y.append(temp2)

        length = [t for t in tr[1] if t !='<PAD>']
        lengths.append(len(length))
        code_labels.append(Variable(torch.LongTensor([int(tr[2])])).cuda() if USE_CUDA else Variable(torch.LongTensor([int(tr[2])])))

    
    train_data = list(zip(train_x,train_y,code_labels))
    
    
    for step in range(STEP):
        for i,(x,y,c) in enumerate(getBatch(BATCH_SIZE,train_data)):

            encoder.zero_grad()
            generator.zero_grad()

            generator_input = Variable(torch.LongTensor([[word2index['<SOS>']]*BATCH_SIZE])).transpose(1,0)

            if USE_CUDA:
                generator_input = generator_input.cuda()

            latent, mu, log_var = encoder(x)

            code = Variable(torch.randn([BATCH_SIZE,2]).uniform_(0,1)).cuda() if USE_CUDA else Variable(torch.randn([BATCH_SIZE,2]).uniform_(0,1))

            score = generator(generator_input,latent,code,lengths,SEQ_LENGTH)
            recon_loss=Recon(score,y.view(-1))
            kld_loss = torch.sum(0.5 * (mu**2 + torch.exp(log_var) - log_var -1))

        #     KL_COST_ANNEALING
            cost_annealing_check = recon_loss.data.cpu().numpy() if USE_CUDA else recon_loss.data.numpy()
            if cost_annealing_check<1.5:
                KTA = 0.5 # KL cost term annealing
            elif cost_annealing_check<1.0:
                KTA = 0.75
            elif cost_annealing_check<0.5:
                KTA = 1.0
            else:
                KTA = 0.0

            ELBO = recon_loss+KTA*kld_loss

            ELBO.backward()

            torch.nn.utils.clip_grad_norm(encoder.parameters(), 5.0)
            torch.nn.utils.clip_grad_norm(generator.parameters(), 5.0)

            gen_optim.step()
            enc_optim.step()

        # KL term Anealing
        #KTA+=1/STEP
        #KTA = round(KTA,3)

        if step % 1==0:
            elbo_for_print = ELBO.data.cpu().numpy() if USE_CUDA else ELBO.data.numpy()
            recon_for_print = recon_loss.data.cpu().numpy() if USE_CUDA else recon_loss.data.numpy()
            kld_for_print = kld_loss.data.cpu().numpy() if USE_CUDA else kld_loss.data.numpy()
            print("[%d/%d] ELBO : %.4f , RECON : %.4f & KLD : %.4f" % (step,STEP,elbo_for_print,
                                                                                  recon_for_print,
                                                                                  kld_for_print))
            elboloss.append(elbo_for_print)
            reconloss.append(recon_for_print)
            kldloss.append(kld_for_print)
    
    
    plt.plot(elboloss)
    plt.plot(reconloss)
    plt.title('ELBO + Recon(NEG)')
    plt.ylabel('Loss')
    plt.xlabel('Step')
    plt.legend(['ElboLoss', 'ReconLoss'], loc='upper left')
    plt.show()
    plt.savefig('ELRCneg' + '.png')
    
    
    plt.plot(kldloss)
    plt.title('KLD(NEG)')
    plt.ylabel('Loss')
    plt.xlabel('Step')
    plt.legend(['KldLoss'], loc='upper left')
    plt.show()
    plt.savefig('KLDneg' + '.png')

    
    
    
    return encoder, generator

In [ ]:
encoder, generatorPOS = TRAIN(trainPOS, encoder, generatorPOS, enc_optim, gen_optimPOS)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:33: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:62: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:63: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.


[0/300] ELBO : 6.2586 , RECON : 6.2586 & KLD : 6619.4072
[1/300] ELBO : 5.8550 , RECON : 5.8550 & KLD : 6281.6895
[2/300] ELBO : 5.6881 , RECON : 5.6881 & KLD : 5047.3018
[3/300] ELBO : 5.2565 , RECON : 5.2565 & KLD : 8277.9893
[4/300] ELBO : 5.0567 , RECON : 5.0567 & KLD : 12604.1797
[5/300] ELBO : 4.6073 , RECON : 4.6073 & KLD : 16336.2266
[6/300] ELBO : 4.3575 , RECON : 4.3575 & KLD : 17415.1738
[7/300] ELBO : 4.2581 , RECON : 4.2581 & KLD : 20223.6387


In [ ]:
encoder, generatorNEG = TRAIN(trainNEG, encoder, generatorNEG, enc_optim, gen_optimNEG)

In [ ]:
encoder, generatorNEU = TRAIN(trainNEU, encoder, generatorNEU, enc_optim, gen_optimNEU)

In [ ]:
# torch.save(encoder.state_dict(),'./encoderPOS.pkl')
# torch.save(generatorPOS.state_dict(),'./generatorPOS.pkl')
# torch.save(generatorNEG.state_dict(),'./generatorNEG.pkl')
# torch.save(generatorNEU.state_dict(),'./generatorNEU.pkl')



## generate


In [ ]:
generator_input = Variable(torch.LongTensor([[word2index['<SOS>']]*1])).transpose(1,0)
if USE_CUDA:
    generator_input = generator_input.cuda()

latent = Variable(torch.randn([1,10])).cuda() if USE_CUDA else Variable(torch.randn([1,10]))
code = Variable(torch.randn([1,2]).uniform_(0,1)).cuda() if USE_CUDA else Variable(torch.randn([1,2]).uniform_(0,1))
recon = generatorNEG(generator_input,latent,code,15,SEQ_LENGTH,False)

v,i = torch.max(recon,1)

decoded=[]
for t in range(i.size()[0]):
    decoded.append(index2word[i.data.cpu().numpy()[t] if USE_CUDA else i.data.cpu().numpy()[t]])
# print(decoded)

print('A: ', ' '.join([i for i in decoded if i !='<PAD>' and i != '<EOS>'])+'\n')

A:  আমরা শুধু ভাবে অযৌক্তিক বন তারা



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:33: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [ ]:
def generate(encoder, generator, text):
    
    t0 = text
    t0 = t0.replace("<br>","")
    t0 = t0.replace("/","")
    token0 = nltk.word_tokenize(t0)
   
    
    if len(token0)>=SEQ_LENGTH:
        token0= token0[:SEQ_LENGTH-1]
    token0.append("<EOS>")

    while len(token0)<SEQ_LENGTH:
        token0.append('<PAD>')
    
    
    temp = prepare_sequence(token0, word2index)
    temp = temp.view(1,-1)
    
    generator_input = Variable(torch.LongTensor([[word2index['<SOS>']]*1])).transpose(1,0)
    if USE_CUDA:
        generator_input = generator_input.cuda()
    
    
    latent, mu, log_var = encoder(temp)
    
    code = Variable(torch.randn([1,2]).uniform_(0,1)).cuda() if USE_CUDA else Variable(torch.randn([1,2]).uniform_(0,1))
    recon = generator(generator_input,latent,code,15,SEQ_LENGTH,False)

    v,i = torch.max(recon,1)

    decoded=[]
    for t in range(i.size()[0]):
        decoded.append(index2word[i.data.cpu().numpy()[t] if USE_CUDA else i.data.cpu().numpy()[t]])
        
    
    print(text)

    print('A: ', ' '.join([i for i in decoded if i !='<PAD>' and i != '<EOS>'])+'\n')
    
    
    

In [ ]:
for i in range(100):
    gen = generate(encoder, generatorNEG, dataPOS[random.randrange(1, 900)])


# nextgen = generate(encoder, generatorPOS, str(gen))
# print(nextgen)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:33: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


এটি বিশাল ভাজা চিংড়ি ছিল যা আমার স্ত্রীর পছন্দ।
A:  এটি দেখায় সময় , গিয়েছিলাম মালিকের প্রথম উদ্যোগ বলে দেখেছি। আদেশ হয়।

গ্রেট bagels পুরানো-ভাবে তৈরির উপায় ।
A:  রেস্টুরেন্টের লোকেরা বেশ অপ্রীতিকর তেমন ছিলনা

আমি এনওয়াইসি দেখি যাতে সুন্দর সুন্দর বলতে পারি!
A:  আমি মনে কয়েকবার এসেছি যে প্রভাবিত আবার হয়ে হবে। পেতেন।

সজ্জা গাঢ়, ঠান্ডা এবং শীতল, যখন খাবারের উপস্থাপনা দর্শনীয়, কম দাম বিবেচনা করে।
A:  খাদ্য রিজার্ভেশন করার চেষ্টা করেছি , বলা হয়েছিল যে তারা সপ্তাহান্ বন্ধ ছিল , শেষ স্মারক দিন ছিল

আমরা সবসময় মনোনিবেশ এবং প্রম্পট সেবা পেয়েছি,পরিবেশ খাঁটি এবং শিথিল হয়।
A:  Chinatown মান দ্বারা মূল্যহীন যদিও , এটি মূল্যহীন , সঙ্কলন বাকি , ।

এই রেস্টুরেন্ট একটি দ্বিতীয় সুযোগ দিন,এটি প্রথম খোলে যখন আপনি সেখানে ছিলেন।
A:  এই জায়গাটি এমন একটি গ্রুপ দ্বারা পরিচালিত হবে যা আসলে গ্রাহক সেবা বুঝতে পারে

আমি এটি একটি ঠান্ডা রাতেপেয়েছি , গরম করার জন্য নিখুঁত স্পট।
A:  এটা দুইবার ক্ষুধার জয়ায় ছিল আনতে 25 ঘন্টা সাথে লাগে না। না। , তারা খাবার পেতেন।

এটা সার্টন প্লেস এর ছাদের উপর নির্ভর ক

In [ ]:
for i in range(200):
    gen = generate(encoder, generatorPOS, dataNEG[random.randrange(1, 900)])



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:33: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


কখনো আমি  এত অমনোযোগী আচরণ করি নি।
A:  যান জন্য ছিল

 সস ব্যবহৃত ছিল  আর তেমন মজাদার ছিলনা
A:  স্ট্যাচু মাংসের সুন্দর গ্রেট করা ? jabbing।

তারা দলগুলোর আসন প্রত্যাখ্যান করে  সপ্তাহান্তে 3 বা তার বেশি বার
A:  একটি এটি ছিল , এই বা জন্য জন্য ।

আমি এখনও পাগল যে আমাকে এই বাজে খাদ্যর জন্য টাকা হয়েছিল
A:  আমি করেছি এবং আমি কিন্তু , না।

অভিভাবকত্ব অভদ্র  হওয়ার অযৌক্তিক বিন্দু।
A:  আপনার খাদ্যের একটি খুব !

প্রকৃত চেহারা তুলনায় ভাল ছিল,তার ওয়েব সাইটে রেস্টুরেন্টের ফটোগুলি ।
A:  দুর্ভাগ্যজনক জায়গা একটি মহান - , , , এবং একটি মাপের সেবা।

আমাদের সবাইকে বের করে দিতে পারত,কেননা গৃহকর্ত্রী এবং ওয়েট্রেস অবিশ্বাস্যভাবে অবাধ্য ছিল।
A:  যে একটি সুস্বাদু , আমি এটি , , , । আমি বলে জন্য ভাল।

 রাতের খাবারের  সময় প্রায় 4 ঘন্টা লেগেছিল কোনও সময়ের ব্যবধানে ছাড়াই।
A:  আমি এর ছিল না এবং খেতে এই ।

এটা যে এর মূল্য টাকার জন্য  নয়
A:  উপভোগ আশ্চর্যজনক না না ছিল ।

পনির ভাল ছিল,পরের বাড়ির grilled এর মধ্যে ।
A:  আমি খাবার খাবার আমি , ডেজার্ট ছিল

দ্বিতীয়ত, এই রাতে জায়গাটি পূর্ব দিকের মহিলার সুগন্ধি দ্ব

In [ ]:
for i in range(100):
    gen = generate(encoder, generatorNEG, dataNEG[random.randrange(1, 900)])



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:33: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


 তারপরও অবশ্যই কোন আসন উপলব্ধ নেই,ওহ হ্যাঁ, এবং তারা ফোন মিথ্যা বলেছে, তারা বাগানে বসা আছে 
A:  আমি গত এর আসন উপলব্ধ নেই , ওহ হ্যাঁ , এবং তারা ফোন মিথ্যা বলেছে , তারা বাগানে বসা আছে

পরিষেবা উন্নত হতে পারে কিন্তু সামগ্রিকভাবে এই জায়গাটি রেস্টুরেন্ট অভিজ্ঞতাতে সামান্য জিনিসগুলির গুরুত্ব (একের জন্য ভারী, কালো, এন্টিক-প্রতারক চায়ের) বোঝে।
A:  এখানে খাওয়াবেন না যদি না আপনি থাই খাবারের জন্য ক্ষুধার্ত থাকেন এবং আপনি পরের দরজায় কাজ করেন।

আমি এটা খাদ্য জন্য একটু দামি বলতে হবে কারণ এটি ভিউ হিসাবে হিসাবে দর্শনীয় ছিল না।
A:  আমি এটা খাদ্য জন্য একটু দামি বলতে হবে কারণ এটি ভিউ হিসাবে হিসাবে দর্শনীয় ছিল না। শুধু দূরে গিয়েছিলা

আমরা একটি টেবিলের জন্য অর্ধেকেরও বেশি সময় অপেক্ষা করতাম (যদিও আমরা রিজার্ভেশন ছিলাম)।
A:  আমরা একটি টেবিলের জন্য অর্ধেকেরও বেশি সময় অপেক্ষা করতাম ( যদিও আমরা রিজার্ভেশন ছিলাম ) ।

অভিভাবকত্ব অভদ্র  হওয়ার অযৌক্তিক বিন্দু।
A:  খাদ্য এটি ছিল - খুব আমি কমপক্ষে খাদ্য ভাল খাওয়ার ছিল করি কিন্তু সব এবং ছিল গন্ধ

সম্ভবত আবার যেতে হবে না ...
A:  এই ভিড় , সুপারিশ , করেছি

এই

In [ ]:
for i in range(100):
    gen = generate(encoder, generatorPOS, dataPOS[random.randrange(1, 900)])



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:33: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


এটি সবসময় ভাল,আদা হাউস এ বেশ কিছু সময় খাওয়া হয়েছে ।
A:  এটি আশ্চর্যজনক প্রিয় ভাল , , মিস রেস্টুরেন্ট রেস্টুরেন্ট । ।

শীতকালে গরম এবং বন্ধুত্বপূর্ণ এবং উষ্ণতর মাসগুলিতে  বহিরঙ্গনে বসে থাকা ভভিতীকর
A:  এটি সুপারিশ স্থান , , , ম্যানহাটানের করা যাবে । ।

আমরা আবার ফিরে যেতে চাই
A:  খুব ভাল এবং অনেক চাই

আমার আন্তরিকভাবে সুপারিশ।
A:  আমার বন্ধু এবং মহান , এটি যা এবং ছিল বড় এর আমরা ) ।

স্টাফ খুব সুবিধাজনক।
A:  খুব ভাল আছে মিষ্টি মধ্যে একটি। এবং ভাল সেবা। ছিল ) ছিল।

পিজা খুব চমত্কার, যেমন হোমডেড পাস্তা।
A:  কম একটি মহান জায়গা ছিল !

যাইহোক তাদের একটি শালীন ওয়াইন তালিকা আছে, খাদ্য ভাল, দাম সঠিক ।
A:  আমাদের আমি একটি একটি একটি এটি , , , আপনি এই আপনি একটি নিঘটকাপের জন্য ।

তোমরা অসাধারণ.
A:  মহান ছিল ছিল , , যখন গ্লাস , ফরাসি ! !

ম্যানহাটানে আমার প্রিয় জায়গাগুলির মধ্যে একটি।
A:  একটি বিশেষ ছিল প্রায়ই , হতে জায়গা একটু যান ।

হর্লির ভয়ঙ্কর!
A:  আমরা চমৎকার বিশাল , দাম অনেক নিখুঁত ?

যদি  আপনি একটি নৈমিত্তিক প্রতিবেশী চান যার ঐ জায়গায় খাবারের দোকান  এবং চমৎকার সেবার ব্যবসথা আছে,।

In [ ]:
generate(encoder, generatorNEG, "মাশরাফি ১৬ কোটি মানুষের ভালোবাসা")

মাশরাফি ১৬ কোটি মানুষের ভালোবাসা
A:  আরে বুড়া মানুষ কামের না।ওরে ভারতের ? ক্যান ? অলআউট জন্য জায়গায় আসতে , আবাল , দেওয়া দেওয়া



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:33: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
